In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Load the dataset
Initially let's load the dataset and check the general conditions in the datset

In [ ]:
dataset = "/kaggle/input/insurance/insurance.csv"
dataset = pd.read_csv(dataset)
dataset.head()

In [ ]:
dataset.info()

In [ ]:
for column in dataset.columns:
    print(column, len(np.unique(dataset[column].to_numpy())))
    

In [ ]:
# Let's convert the categorical data into numerical
dataset["sex"] = dataset["sex"].map({"female":-1, "male":+1})
dataset["smoker"] = dataset["smoker"].map({"no":-1, "yes":+1})

In [ ]:
dataset.iloc[:, ]

In [ ]:
# Converting the region categorical value to numerical using the (c-1) effect-encoding
dataset_y = dataset.iloc[:, -1]
dataset_x = pd.get_dummies(dataset.iloc[:, :-1], columns=["region"])
dataset_x.info()

In [ ]:
dataset_x.head()

In [ ]:
import torch
from torch import Tensor
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

pivot = int(len(dataset_x) * (0.8))

In [ ]:
milad = np.array([[1,2],[2,3]])
rahim = np.array([9, 10])
np.concatenate([milad, rahim[:, None]], axis=1)

In [ ]:
from sklearn.preprocessing import StandardScaler
# Normalize the dataset

x_train = dataset_x.to_numpy()[:pivot, :]
y_train = dataset_y.to_numpy()[:pivot]

x_eval = dataset_x.to_numpy()[pivot:, :]
y_eval = dataset_y.to_numpy()[pivot:]

sc = StandardScaler()
temp = sc.fit_transform(np.concatenate([x_train, y_train[:, None]], axis=1))
x_train, y_train = Tensor(temp[:, :-1]), Tensor(temp[:, -1])

temp = sc.transform(np.concatenate([x_eval, y_eval[:, None]], axis=1))
x_eval, y_eval = Tensor(temp[:, :-1]), Tensor(temp[:, -1])

train_data = TensorDataset(x_train, y_train)
eval_data = TensorDataset(x_eval, y_eval)

In [ ]:
train_loader = DataLoader(train_data,
                       batch_size = 128)
eval_loader = DataLoader(eval_data, 
                        batch_size = 64)

## Building the model
At the moment we will try to build the model. It is simply a FFNN(Fed forward neyral network)

In [ ]:
input_size = 9
epochs = 5

In [ ]:
layers = []
layers.append(nn.Linear(input_size, 1000))
layers.append(nn.BatchNorm1d(1000))
layers.append(nn.ReLU())
layers.append(nn.Linear(1000, 1000))
layers.append(nn.ReLU())
layers.append(nn.Linear(1000, 1000))
layers.append(nn.ReLU())
layers.append(nn.Linear(1000, 1))
model = nn.Sequential(*layers)
print(model)

In [ ]:
criterian = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())
train_losses = []
eval_losses = []

In [ ]:
for epoch in range(epochs):
    model.train()
    temp = []
    for x_train, y_train in train_loader:
        # Forward
        out = model(x_train)
        loss = criterian(y_train, out)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        temp.append(loss.item())
    
    train_losses.append(np.mean(temp))
    temp = []
    model.eval()
    for x_eval, y_eval in eval_loader:
        out = model(x_eval)
        temp.append(criterian(y_eval, out).item())
    eval_losses.append(np.mean(temp))
        
    print(f"({epoch+1:2}/{epochs}): train_loss: {train_losses[-1]:.4f}, eval_loss: {eval_losses[-1]:.4f}")

In [ ]:
from matplotlib import pyplot as plt
plt.plot(list(range(epochs)), train_losses)
plt.plot(list(range(len(eval_losses))), eval_losses)
# plt.ylim(0.8, 1.1)
plt.grid(alpha=0.5)
plt.legend(["train_loss", "eval_loss"])
plt.show()